In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [32]:
def preprocessing(df, user_core=20, threshold=3.5):
    profile_size = df.groupby("user_id").size()
    uids = profile_size[profile_size >= user_core].index
    modified_df = df[df["user_id"].isin(uids)].copy()
    modified_df = modified_df[modified_df["rating"].astype(float) >= threshold]
    return modified_df

# Amazon

In [275]:
df = pd.read_csv("Grocery_and_Gourmet_Food.csv", sep=",", header=None, names=["item_id", "user_id", "rating", "timestamp"])
df = df[["user_id", "item_id", "rating"]]
df["user_id"] = df["user_id"].map({b: a for a, b in enumerate(df["user_id"].unique())})
df["item_id"] = df["item_id"].map({b: a for a, b in enumerate(df["item_id"].unique())})
df.head()

,user_id,item_id,rating
0,0,0,5.0
1,1,0,4.0
2,2,0,4.0
3,3,0,5.0
4,4,0,4.0


In [276]:
thresh = df["rating"].mean()
item_core = 5
user_core = 20
modified_df = df.copy()
n_prev_interactions = -1
stop = False
while stop is False:
    # item core
    item_profile_size = modified_df.groupby("item_id").size()
    iids = item_profile_size[item_profile_size >= item_core].index
    modified_df = modified_df[modified_df["item_id"].isin(iids)].copy()

    # user core
    profile_size = modified_df.groupby("user_id").size()
    uids = profile_size[profile_size >= user_core].index
    modified_df = modified_df[modified_df["user_id"].isin(uids)].copy()
    #modified_df = modified_df[modified_df["rating"].astype(float) >= thresh].copy()

    if n_prev_interactions == len(modified_df):
        stop = True
        print()
        print("Converged ... apply rating threshold")
        modified_df = modified_df[modified_df["rating"].astype(float) >= thresh].copy()
        print(modified_df["user_id"].nunique(), modified_df["item_id"].nunique(), len(modified_df), modified_df.groupby("user_id").size().mean(), modified_df.groupby("item_id").size().mean(), 100 * (len(modified_df) / (modified_df["user_id"].nunique() * modified_df["item_id"].nunique())))

    else:
        n_prev_interactions = len(modified_df)
        print(modified_df["user_id"].nunique(), modified_df["item_id"].nunique(), len(modified_df), modified_df.groupby("user_id").size().mean(), modified_df.groupby("item_id").size().mean(), 100 * (len(modified_df) / (modified_df["user_id"].nunique() * modified_df["item_id"].nunique())))

final_df = modified_df.copy()

8454 58532 288374 34.11095339484268 4.926775097382628 0.05827744378261922
4482 12658 147676 32.94868362338242 11.666613999051982 0.2602992860118693
3672 8565 120435 32.798202614379086 14.061295971978984 0.3829328968403863
3412 7503 111681 32.73182883939039 14.88484606157537 0.4362498845713766
3318 7131 108496 32.69921639541893 15.21469639601739 0.4585502229058887
3276 6994 107161 32.71092796092796 15.321847297683728 0.4676998564616523
3260 6945 106666 32.71963190184049 15.358675305975522 0.4711250093857522
3255 6921 106476 32.71152073732719 15.384482011270048 0.47264153644454826
3252 6908 106367 32.70817958179582 15.397654892877823 0.4734826227822209
3250 6905 106319 32.71353846153846 15.39739319333816 0.4737659444104049
3249 6899 106276 32.71037242228378 15.404551384258589 0.47413208323356687
3247 6895 106222 32.71388974437943 15.405656272661348 0.47445815437823674
3244 6890 106145 32.72040690505548 15.40566037735849 0.47489705232301144
3244 6885 106125 32.714241676942045 15.413943355

In [277]:
final_df.groupby("user_id").size().describe()

count    3222.000000
mean       22.400993
std        15.411482
min         1.000000
25%        15.000000
50%        20.000000
75%        26.000000
max       230.000000
dtype: float64

In [278]:
final_df["user_id"].nunique()

3222

In [279]:
final_df[["user_id", "item_id"]].to_csv("grocery_preprocessed.csv", index=False, header=None)

# Amazon Sports and Outdoors

In [37]:
df = pd.read_csv("sportsandoutdoors/Sports_and_Outdoors.csv", sep=",", header=None, names=["item_id", "user_id", "rating", "timestamp"])
df = df[["user_id", "item_id", "rating"]]
df["user_id"] = df["user_id"].map({b: a for a, b in enumerate(df["user_id"].unique())})
df["item_id"] = df["item_id"].map({b: a for a, b in enumerate(df["item_id"].unique())})
df.head()

,user_id,item_id,rating
0,0,0,5.0
1,1,0,4.0
2,2,0,1.0
3,3,0,5.0
4,4,0,1.0


In [57]:
ratings_df = df
print(ratings_df["user_id"].nunique(), ratings_df["item_id"].nunique(), len(ratings_df), ratings_df.groupby("user_id").size().mean())
print(ratings_df["rating"].mean())
final_df = preprocessing(ratings_df, user_core=20, threshold=3.5)
#final_df = preprocessing(ratings_df, user_core=20, threshold=ratings_df["rating"].mean())
print(final_df["user_id"].nunique(), final_df["item_id"].nunique(), len(final_df), final_df.groupby("user_id").size().mean(), final_df.groupby("item_id").size().mean(), 100 * (len(final_df) / (final_df["user_id"].nunique() * final_df["item_id"].nunique())))

6703391 957764 12980837 1.9364582791008311
4.243398403354114
22703 172060 602952 26.558252213363872 3.504312449145647 0.015435459847357825


In [121]:
# filter items
item_core = 25
item_profile_size = df.groupby("item_id").size()
iids = item_profile_size[item_profile_size >= item_core].index
modified_df = df[df["item_id"].isin(iids)].copy()

final_df = preprocessing(modified_df, user_core=20, threshold=modified_df["rating"].mean())
print(final_df["user_id"].nunique(), final_df["item_id"].nunique(), len(final_df), final_df.groupby("user_id").size().mean(), final_df.groupby("item_id").size().mean(), 100 * (len(final_df) / (final_df["user_id"].nunique() * final_df["item_id"].nunique())))

11959 48398 242643 20.289572706748057 5.0134922930699615 0.041922337093987475


In [162]:
df["rating"].mean()

4.243398403354114

In [177]:
thresh = df["rating"].mean()
item_core = 2
user_core = 20
modified_df = df.copy()
n_prev_interactions = -1
stop = False
while stop is False:
    # item core
    item_profile_size = modified_df.groupby("item_id").size()
    iids = item_profile_size[item_profile_size >= item_core].index
    modified_df = modified_df[modified_df["item_id"].isin(iids)].copy()

    # user core
    profile_size = modified_df.groupby("user_id").size()
    uids = profile_size[profile_size >= user_core].index
    modified_df = modified_df[modified_df["user_id"].isin(uids)].copy()
    #modified_df = modified_df[modified_df["rating"].astype(float) >= thresh].copy()

    if n_prev_interactions == len(modified_df):
        stop = True
        print()
        print("Converged ... apply rating threshold")
        modified_df = modified_df[modified_df["rating"].astype(float) >= thresh].copy()
        print(modified_df["user_id"].nunique(), modified_df["item_id"].nunique(), len(modified_df), modified_df.groupby("user_id").size().mean(), modified_df.groupby("item_id").size().mean(), 100 * (len(modified_df) / (modified_df["user_id"].nunique() * modified_df["item_id"].nunique())))

    else:
        n_prev_interactions = len(modified_df)
        print(modified_df["user_id"].nunique(), modified_df["item_id"].nunique(), len(modified_df), modified_df.groupby("user_id").size().mean(), modified_df.groupby("item_id").size().mean(), 100 * (len(modified_df) / (modified_df["user_id"].nunique() * modified_df["item_id"].nunique())))

final_df = modified_df.copy()

21374 168017 657044 30.740338729297278 3.9105804769755443 0.018295969294355497
15031 75630 460248 30.61991883440889 6.085521618405394 0.04048647208040313
14079 63659 430703 30.591874422899355 6.765783314221085 0.048055851368854915
13862 61165 424141 30.597388544221612 6.934374233630344 0.05002434160749058
13807 60585 422522 30.602013471427536 6.974036477675992 0.05051087475683344
13796 60419 422147 30.599231661351116 6.986990847250037 0.05064504818244446
13793 60392 422065 30.599941999564997 6.988756788978673 0.050668866736595895
13793 60388 422061 30.59965199738998 6.989153474200172 0.05067174272602169

Converged ... apply rating threshold
13736 56446 287237 20.911255096097847 5.088704248308117 0.037046478220064925


In [178]:
final_df.groupby("user_id").size().describe()

count    13736.000000
mean        20.911255
std         12.597689
min          1.000000
25%         14.000000
50%         19.000000
75%         25.000000
max        247.000000
dtype: float64

In [155]:
final_df[["user_id", "item_id"]].to_csv("sportsoutdoors_preprocessed.csv", index=False, header=None)

# LFM 3k User Groups

In [22]:
df = pd.read_csv("LFM-3k/artist_ratings.csv", sep="\;", header=None, names=["user_id", "item_id", "listening_count"])
df.head()

C:\Users\pmuellner\AppData\Local\Temp\ipykernel_5436\3753643500.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("LFM-3k/artist_ratings.csv", sep="\;", header=None, names=["user_id", "item_id", "listening_count"])


,user_id,item_id,listening_count
0,1021445,12,184.222707
1,1021445,16,1.000000
2,1021445,28,27.174672
3,1021445,29,1.000000
4,1021445,46,1.000000


In [23]:
ratings_df = pd.DataFrame()
for user_id, data in df.groupby("user_id"):
    ratings = MinMaxScaler(feature_range=(1, 5)).fit_transform(data["listening_count"].values.reshape(-1, 1).astype(float))
    new_rows = data[["user_id", "item_id"]].copy()
    new_rows["rating"] = ratings
    ratings_df = pd.concat([ratings_df, new_rows])
ratings_df.columns = ["user_id", "item_id", "rating"]

print(ratings_df["user_id"].nunique(), ratings_df["item_id"].nunique(), len(ratings_df), ratings_df.groupby("user_id").size().mean())
print(ratings_df["rating"].mean())

final_df = preprocessing(ratings_df, user_core=20, threshold=ratings_df["rating"].mean())
print(final_df["user_id"].nunique(), final_df["item_id"].nunique(), len(final_df), final_df.groupby("user_id").size().mean(), final_df.groupby("item_id").size().mean(), 100 * (len(final_df) / (final_df["user_id"].nunique() * final_df["item_id"].nunique())))

3000 352805 1755361 585.1203333333333
1.1292309186701142
2999 78799 348437 116.18439479826608 4.421845454891559 0.14744399649521706


In [24]:
final_df[["user_id", "item_id"]].to_csv("lfm3k_preprocessed.csv", index=False, header=None)

# ML 1M

In [265]:
df = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None)
df.columns = ["user_id", "item_id", "rating", "timestamp"]
df.head()

C:\Users\pmuellner\AppData\Local\Temp\ipykernel_5436\3472538835.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None)


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [266]:
ratings_df = df
print(ratings_df["user_id"].nunique(), ratings_df["item_id"].nunique(), len(ratings_df), ratings_df.groupby("user_id").size().mean(), ratings_df.groupby("item_id").size().mean(), 100 * (len(ratings_df) / (ratings_df["user_id"].nunique() * ratings_df["item_id"].nunique())))
print(ratings_df["rating"].mean())

final_df = preprocessing(ratings_df, user_core=20, threshold=ratings_df["rating"].mean())
print(final_df["user_id"].nunique(), final_df["item_id"].nunique(), len(final_df), final_df.groupby("user_id").size().mean(), final_df.groupby("item_id").size().mean(), 100 * (len(final_df) / (final_df["user_id"].nunique() * final_df["item_id"].nunique())))

6040 3706 1000209 165.5975165562914 269.88909875876953 4.468362562231285
3.581564453029317
6038 3533 575281 95.27674726730706 162.8307387489386 2.6967661270112386


In [267]:
final_df.groupby("user_id").size().describe()

count    6038.000000
mean       95.276747
std       105.005005
min         1.000000
25%        27.000000
50%        58.000000
75%       124.000000
max      1435.000000
dtype: float64

In [30]:
final_df[["user_id", "item_id"]].to_csv("ml1m_preprocessed.csv", index=False, header=None)